# Change Detection with DEMs

## Introduction

## Study Area

In [1]:
from pathlib import Path
import geopandas as gpd

# Create a GeoDataFrame from the GeoJson aoi file adjacent to this notebook
AOI_GEOJSON = Path(__name__).resolve().parent / "aoi.geojson"
aoi: gpd.GeoDataFrame = gpd.read_file(AOI_GEOJSON)

In [2]:
aoi.head()

,name,geometry
0,Area of Interest,"POLYGON ((-147.12428 61.12111, -147.09331 61.1..."


In [3]:
import xyzservices.providers as xyz
import folium

# Define basemap options using providers from the xyzservices package
# A full list of providers is available in the package docs: https://xyzservices.readthedocs.io/en/stable/introduction.html
TILE_PROVIDERS = {
    "ESRI WorldImagery": xyz.Esri.WorldImagery,
    "USGS USImagery": xyz.USGS.USImagery,
}


def explore_area_of_interest(gdf: gpd.GeoDataFrame) -> folium.Map:
    explore_kwargs = {
        "tiles": None,  # Prevents OpenStreetMap from being added as a basemape layer
        "column": "name",  # Can be any column in the GeoDataFrame besides geometry, if this isn't specified, style settings are ignored
        "style_kwds": {  # Style the vectors as red outlines
            "fill": False,
            "color": "red",
        },
        "legend": False,  # Hide the default legend
    }

    # Initialize the Map object from the provided GeoDataFrame
    m: folium.Map = gdf.explore(**explore_kwargs)
    # Add the basemaps defined in the TILE_PROVIDERS constant
    for name, provider in TILE_PROVIDERS.items():
        folium.TileLayer(tiles=provider, name=name).add_to(m)
    # Enable the layer controls for switching basemaps
    folium.LayerControl().add_to(m)
    return m

In [4]:
aoi_map = explore_area_of_interest(aoi)
aoi_map

## Backgound Data

## Finding Strip DEMs

## Prepare Strip DEMs for Analysis

## Visualizing Change with Differencing